# Multilayer Perceptrons

In [44]:
import io
import requests
import logging
import itertools
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (mean_squared_error, r2_score,
                             confusion_matrix, ConfusionMatrixDisplay)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Restrict tensorflow output to errors
logging.getLogger('tensorflow').setLevel(logging.ERROR)

# Make notebook output stable across runs
random_state = 100
np.random.seed(random_state)

# Options for plots
%matplotlib inline
sns.set()
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.labelpad'] = 12

## Regression: Revenue Forecasting

In [102]:
df = pd.read_csv('../flights_sample_3m.csv')
df.head()

,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",...,0.0,186.0,176.0,153.0,1065.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",...,0.0,118.0,112.0,87.0,680.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
4,2020-02-23,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,407,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",...,0.0,181.0,182.0,153.0,985.0,NaN,NaN,NaN,NaN,NaN


In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 32 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   FL_DATE                  object 
 1   AIRLINE                  object 
 2   AIRLINE_DOT              object 
 3   AIRLINE_CODE             object 
 4   DOT_CODE                 int64  
 5   FL_NUMBER                int64  
 6   ORIGIN                   object 
 7   ORIGIN_CITY              object 
 8   DEST                     object 
 9   DEST_CITY                object 
 10  CRS_DEP_TIME             int64  
 11  DEP_TIME                 float64
 12  DEP_DELAY                float64
 13  TAXI_OUT                 float64
 14  WHEELS_OFF               float64
 15  WHEELS_ON                float64
 16  TAXI_IN                  float64
 17  CRS_ARR_TIME             int64  
 18  ARR_TIME                 float64
 19  ARR_DELAY                float64
 20  CANCELLED                float64
 21  CANCELLA

In [104]:
df['AIRLINE'].value_counts()

Southwest Airlines Co.                576470
Delta Air Lines Inc.                  395239
American Airlines Inc.                383106
SkyWest Airlines Inc.                 343737
United Air Lines Inc.                 254504
Republic Airline                      143107
Envoy Air                             121256
JetBlue Airways                       112844
Endeavor Air Inc.                     112463
PSA Airlines Inc.                     107050
Alaska Airlines Inc.                  100467
Spirit Air Lines                       95711
Mesa Airlines Inc.                     65012
Frontier Airlines Inc.                 64466
Allegiant Air                          52738
Hawaiian Airlines Inc.                 32114
Horizon Air                            20634
ExpressJet Airlines LLC d/b/a aha!     19082
Name: AIRLINE, dtype: int64

In [105]:
# Keeping the top 15 airlines. Was thinking 10, but 4 of the next 5 are quite popular airlines
# So decided to do 15
AIRLINE_DROPS = df['AIRLINE'].value_counts().head(15)

In [106]:
# Dropping the airlines that are not in the top 15
drop_rows = df[~df['AIRLINE'].isin(AIRLINE_DROPS.index)].index
df.drop(drop_rows, inplace=True)

In [107]:
# We have so many airports that when we created dummy variables, the ~700 features along with 3 million rows
# created a csv that was 5GB, and trying to import it into a df would crash my interpreter.
# First tried dropping just the bottom 24 (under 100 flights) however thinking we will just keep the
# 100 most popular
# Still dealing with some performance issues and crashes at 100 (although better)
# Changing to top 75. Top 100 still had us with 2.5million, so haven't lost much
ORIGIN_DROPS = df['ORIGIN'].value_counts().head(75)

In [108]:
# Drop rows that are not in the top 75
drop_rows = df[~df['ORIGIN'].isin(ORIGIN_DROPS.index)].index
df.drop(drop_rows, inplace=True)

In [109]:
# Repeat for Destination
DEST_DROPS = df['DEST'].value_counts().head(75)

In [110]:
# Drop rows that are not in the top 75 destinations
drop_rows = df[~df['DEST'].isin(DEST_DROPS.index)].index
df.drop(drop_rows, inplace=True)

In [111]:
df.shape

(2167344, 32)

In [112]:
df.head()

,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",...,0.0,186.0,176.0,153.0,1065.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",...,0.0,118.0,112.0,87.0,680.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
4,2020-02-23,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,407,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",...,0.0,181.0,182.0,153.0,985.0,NaN,NaN,NaN,NaN,NaN


In [113]:
# Before we try to predict delay/cancellation with the model, we want to decide which features
# we would have availible to us while booking a flight, as it will not be able to generalize
# if it trains on data collected after the flight

df = df[['FL_DATE', 'AIRLINE_CODE', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'CRS_ELAPSED_TIME', 
        'DISTANCE', 'CANCELLED', 'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS',
        'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT']]

Some columns could be consolidated as well. Use airline code for indentifying airlines and drop the other 2. Use origin and Dest columns and can remove city due to redundancy 

Note that CRS times are the scheduled times, which is why that is what we will use for our model.

Shouldn't need CRS_ARR_TIME since it is directly related to CRS_DEP_TIME and CRS_ELAPSED_TIME which are already in model

Data dictionary says the distance is between the airports, not distance traveled (which would be greater when diverted and thus show correlation with delays), so will use it.

In [114]:
df.head()

,FL_DATE,AIRLINE_CODE,ORIGIN,DEST,CRS_DEP_TIME,CRS_ELAPSED_TIME,DISTANCE,CANCELLED,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,UA,FLL,EWR,1155,186.0,1065.0,0.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,DL,MSP,SEA,2120,235.0,1399.0,0.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,UA,DEN,MSP,954,118.0,680.0,0.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,DL,MSP,SFO,1609,260.0,1589.0,0.0,0.0,0.0,24.0,0.0,0.0
4,2020-02-23,NK,MCO,DFW,1840,181.0,985.0,0.0,NaN,NaN,NaN,NaN,NaN


In [115]:
df.shape

(2167344, 13)

In [116]:
# Consolidate all the delay columns into one delayed column to use as our y
df['DELAYED_TIME'] = df['DELAY_DUE_CARRIER'] + df['DELAY_DUE_WEATHER'] + df['DELAY_DUE_NAS'] + df['DELAY_DUE_SECURITY'] + df['DELAY_DUE_LATE_AIRCRAFT']

In [117]:
# Create binary classifier for delayed flights
#df['DELAYED'] = df['DELAYED'].mask(df['DELAYED_TIME'] > 0, 1)
df['DELAYED'] = df['DELAYED_TIME'].map(lambda x: 1 if x > 0 else 0)

In [118]:
df = df.drop(columns=['DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT'])

In [119]:
# Map FL_DATE to months columns
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])
df['MONTH'] = df['FL_DATE'].dt.month
# df_subset = df_subset.drop(columns=['FL_DATE'])

In [120]:
# Get day of the week column
df['DAY_OF_WEEK'] = df['FL_DATE'].dt.dayofweek

In [121]:
df.head()

,FL_DATE,AIRLINE_CODE,ORIGIN,DEST,CRS_DEP_TIME,CRS_ELAPSED_TIME,DISTANCE,CANCELLED,DELAYED_TIME,DELAYED,MONTH,DAY_OF_WEEK
0,2019-01-09,UA,FLL,EWR,1155,186.0,1065.0,0.0,NaN,0,1,2
1,2022-11-19,DL,MSP,SEA,2120,235.0,1399.0,0.0,NaN,0,11,5
2,2022-07-22,UA,DEN,MSP,954,118.0,680.0,0.0,NaN,0,7,4
3,2023-03-06,DL,MSP,SFO,1609,260.0,1589.0,0.0,24.0,1,3,0
4,2020-02-23,NK,MCO,DFW,1840,181.0,985.0,0.0,NaN,0,2,6


In [133]:
df.info()

<bound method DataFrame.info of          CRS_DEP_TIME  CRS_ELAPSED_TIME  DISTANCE  CANCELLED  DELAYED_TIME  \
0                 715             186.0    1065.0        0.0           0.0   
1                1280             235.0    1399.0        0.0           0.0   
2                 594             118.0     680.0        0.0           0.0   
3                 969             260.0    1589.0        0.0          24.0   
4                1120             181.0     985.0        0.0           0.0   
...               ...               ...       ...        ...           ...   
2999992           778             132.0     818.0        0.0           0.0   
2999994           420             164.0    1076.0        0.0           0.0   
2999995          1062              85.0     328.0        0.0           0.0   
2999996           780             176.0     977.0        0.0           0.0   
2999998           960              88.0     184.0        0.0           0.0   

         DELAYED  AIRLINE_CODE_

In [122]:
# Create dummy variables to use in our model for categorical columns
df = pd.get_dummies(df, columns=['AIRLINE_CODE', 'ORIGIN', 'DEST', 'MONTH', 'DAY_OF_WEEK'])

In [123]:
df.head()

,FL_DATE,CRS_DEP_TIME,CRS_ELAPSED_TIME,DISTANCE,CANCELLED,DELAYED_TIME,DELAYED,AIRLINE_CODE_9E,AIRLINE_CODE_AA,AIRLINE_CODE_AS,...,MONTH_10,MONTH_11,MONTH_12,DAY_OF_WEEK_0,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6
0,2019-01-09,1155,186.0,1065.0,0.0,NaN,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2022-11-19,2120,235.0,1399.0,0.0,NaN,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,2022-07-22,954,118.0,680.0,0.0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2023-03-06,1609,260.0,1589.0,0.0,24.0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,2020-02-23,1840,181.0,985.0,0.0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [124]:
df.shape

(2167344, 191)

In [125]:
# Drop the FL_DATE column
df.drop(columns=['FL_DATE'], inplace=True)

In [126]:
df.columns[df.isna().any()].tolist()

['CRS_ELAPSED_TIME', 'DELAYED_TIME']

In [127]:
# Drop the rows with null scheduled flight time
df = df.dropna(subset=['CRS_ELAPSED_TIME'])

In [128]:
df['DELAYED_TIME'] = df['DELAYED_TIME'].fillna(0)

In [129]:
# Convert scheduled depature time from HHMM to minutes
df['CRS_DEP_TIME'] = df['CRS_DEP_TIME'] // 100 * 60 + df['CRS_DEP_TIME'] % 100

In [130]:
df.head()

,CRS_DEP_TIME,CRS_ELAPSED_TIME,DISTANCE,CANCELLED,DELAYED_TIME,DELAYED,AIRLINE_CODE_9E,AIRLINE_CODE_AA,AIRLINE_CODE_AS,AIRLINE_CODE_B6,...,MONTH_10,MONTH_11,MONTH_12,DAY_OF_WEEK_0,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6
0,715,186.0,1065.0,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1280,235.0,1399.0,0.0,0.0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,594,118.0,680.0,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,969,260.0,1589.0,0.0,24.0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1120,181.0,985.0,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [131]:
df.shape

(2167342, 190)

In [132]:
# Export our data to a csv file so we do not need to redo above steps each time we open file
df.to_csv('../flights_sample_3m_cleaned.csv', index=False)